In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [5]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="bitcoin-transaction-2-c7d551f5f41d.json"
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

query = """
SELECT 
  value AS number_ether,
  from_address AS input,
  to_address AS output,
  receipt_gas_used AS gas_used,
  DATE(timestamp) AS tx_date
FROM
  `bigquery-public-data.crypto_ethereum.transactions` AS transactions,
  `bigquery-public-data.crypto_ethereum.blocks` AS blocks
WHERE TRUE
  AND transactions.block_number = blocks.number
  AND receipt_status = 1

GROUP BY number_ether, input, output, tx_date, gas_used
HAVING tx_date >= '2021-06-01' AND tx_date <= '2021-06-29'
ORDER BY tx_date
LIMIT 30000
"""

In [6]:
query_job = client.query(query)

iterator = query_job.result(timeout=300)
rows = list(iterator)

# Transform the rows into a nice pandas dataframe
df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))

# Look at the first 10
df.head(10)

,number_ether,input,output,gas_used,tx_date
0,60000000000000000,0xf31a14fb60253aa43180e83ec0e1ef55317375f5,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,678989,2021-06-01
1,0,0x6084b0d83b74a22dffb57e9e44af764214f73776,0x06a01a4d579479dd5d884ebf61a31727a3d8d442,19321,2021-06-01
2,0,0x04391832fb8259b7937c5dc7c83b1ff1329feb4f,0x106552c11272420aad5d7e94f8acab9095a6c952,46625,2021-06-01
3,0,0x9faf7eb62a9e085da006ab7ef288acd3fc001abb,0xdac17f958d2ee523a2206206994597c13d831ec7,48897,2021-06-01
4,102897000000000000,0xbf40df4000e97724de9ce836d10b5ba6a497c7ab,0x4960a1b0b0538bdee1551ec82dc27274edb4d03c,21000,2021-06-01
5,1610609000000000,0x792134291def01b9f0d34db714a2711938cfef93,0xd3242291d3ee6d38a1ff101fabfe59a1c602d3dd,21000,2021-06-01
6,2400000000000000,0x03599a2429871e6be1b154fb9c24691f9d301865,0x126f5eaebe1e3718a746081ffc5d32c6d0a0edd4,21000,2021-06-01
7,30000000000000000,0x6423c353874c1da5f52ddca50478255c82ca9af3,0x7a250d5630b4cf539739df2c5dacb4c659f2488d,332083,2021-06-01
8,53373570562693141,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0x11fb2155155ebc6c6d7f101868380610450c0230,21000,2021-06-01
9,1698969180000000000,0x193dfad2eaf2a5d6c0a2ec8ec2d1cb698d835fa8,0x6262998ced04146fa42253a5c0af90ca02dfd2a3,21000,2021-06-01


In [7]:
df.to_csv("BigQuery_Ethereum_Dataset.csv", index=False)